In [1]:

from tandv.track.common import read_pickle, TrainingStats
from tandv.viz import exp_hist,scalar_global_heatmap,scalar_line,interactive
import pickle

## Loading Example Data

In [3]:
lf_dir = '/home/ubuntu/numerics-vis/local-explorations/profiling-logging-overhead/numerics-llama.c/tandv/plain-pocket/final_logframe.pkl'
ts_dir = '/home/ubuntu/numerics-vis/local-explorations/profiling-logging-overhead/numerics-llama.c/training_stats.pkl'
with open('/home/ubuntu/numerics-vis/local-explorations/profiling-logging-overhead/numerics-llama.c/training_stats.pkl', 'rb') as f:
    dct = pickle.load(f)
# Need to provide a url to some example data
lf = read_pickle(lf_dir)
tstats = TrainingStats(
    steps=dct['step'],
    train_loss=dct['train_loss'],
    val_loss=dct['val_loss']
)



## Interactive Exponent Histogram
Pass `exp_hist`function into `interactive` along with valid arguements to generate an initial plot.

You can then use the various widgets to query along different dimensions (step, layer, tensor type, etc..) and subsequently generate visualizations.

In [2]:
interactive(exp_hist,
    df=lf,
    layer='output',
    tt = 'Activation',
    step = 0,
    dtype_annotation = 'float8_e4m3fn'
)

NameError: name 'lf' is not defined

## Interactive Scalar Line 
Pass `scalar_line`function into `interactive` along with valid arguements to generate an initial plot.

You can then use the various widgets to query along different dimensions (scalar_metric, layer, tensor type, etc..) and subsequently generate visualizations.

In [6]:
interactive(
    scalar_line,
    df=lf,
    layer='output',
    tt='Gradient',
    scalar_metric='std',
)

Output(layout=Layout(width='100%'), outputs=({'output_type': 'display_data', 'data': {'text/plain': "HBox(chil…

Output(layout=Layout(overflow='scroll hidden', width='1500px'), outputs=({'output_type': 'display_data', 'data…

## Interactive Global Heatmap 
Pass `scalar_global_heatmap`function into `interactive` along with valid arguements to generate an initial plot.

You can then use the various widgets to query along different dimensions (scalar_metric, layer, tensor type, etc..) and subsequently generate visualizations.

You can also click on the heatmap patches and it will generate an `exp_hist` vizualization along those query dimensions (step, tensor_type and layer)

In [7]:
interactive(
    scalar_global_heatmap,
    df=lf,
    tt='Activation',
    scalar_metric='std',
    inc=5,
)

Output(layout=Layout(width='100%'), outputs=({'output_type': 'display_data', 'data': {'text/plain': "HBox(chil…

Output(layout=Layout(overflow='scroll hidden', width='1500px'), outputs=({'output_type': 'display_data', 'data…

# Cross Referencing with Training Stats

Query exponent histograms from the loss curve

In [4]:
interactive(exp_hist,
    train_stats=tstats,
    df=lf,
    layer=[n for n in lf.metadata.name.unique().tolist() if 'layers.5.feed_forward.w' in n],
    tt = 'Activation',
    step = 0,
    dtype_annotation = 'float8_e4m3fn',
    col_wrap = 3
)

Cross referencing loss curve(s) with scalar statistics of various tensors in the network

In [4]:
interactive(
    scalar_line,
    train_stats=tstats,
    df=lf,
    layer=[n for n in lf.metadata.name.unique().tolist() if 'layers.3.attention.w' in n],
    tt='Gradient',
    scalar_metric=['std','rm2'],
    col_wrap = 3
)